In [2]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import datetime
import warnings

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('년,월별 총 범죄데이터.csv', encoding='cp949', index_col=0)
df.head()

,20년1월_범죄건수,21년1월_범죄건수,22년1월_범죄건수,20년2월_범죄건수,21년2월_범죄건수,22년2월_범죄건수,20년3월_범죄건수,21년3월_범죄건수,22년3월_범죄건수,20년4월_범죄건수,...,22년9월_범죄건수,20년10월_범죄건수,21년10월_범죄건수,22년10월_범죄건수,20년11월_범죄건수,21년11월_범죄건수,22년11월_범죄건수,20년12월_범죄건수,21년12월_범죄건수,22년12월_범죄건수
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,715,438,569,508,338,398,679,487,518,604,...,549,519,548,580,592,634,706,606,518,630
강동구,332,242,324,287,196,222,283,265,244,264,...,298,310,241,270,312,375,450,292,344,303
강북구,258,141,217,242,128,152,258,167,204,252,...,244,212,201,240,182,227,241,183,191,214
강서구,443,258,353,373,223,262,421,286,357,330,...,368,377,359,471,297,387,434,289,402,343
관악구,466,304,378,365,287,284,396,331,310,456,...,329,469,296,395,431,470,600,461,450,461


In [9]:
df.columns

Index(['20년1월_범죄건수', '21년1월_범죄건수', '22년1월_범죄건수', '20년2월_범죄건수', '21년2월_범죄건수',
       '22년2월_범죄건수', '20년3월_범죄건수', '21년3월_범죄건수', '22년3월_범죄건수', '20년4월_범죄건수',
       '21년4월_범죄건수', '22년4월_범죄건수', '20년5월_범죄건수', '21년5월_범죄건수', '22년5월_범죄건수',
       '20년6월_범죄건수', '21년6월_범죄건수', '22년6월_범죄건수', '20년7월_범죄건수', '21년7월_범죄건수',
       '22년7월_범죄건수', '20년8월_범죄건수', '21년8월_범죄건수', '22년8월_범죄건수', '20년9월_범죄건수',
       '21년9월_범죄건수', '22년9월_범죄건수', '20년10월_범죄건수', '21년10월_범죄건수', '22년10월_범죄건수',
       '20년11월_범죄건수', '21년11월_범죄건수', '22년11월_범죄건수', '20년12월_범죄건수',
       '21년12월_범죄건수', '22년12월_범죄건수'],
      dtype='object')

In [10]:
pattern = ['20년', '21년', '22년']
months = [f'{month}월' for month in range(1, 13)]
sorted_columns = [f'{year}{month}_범죄건수' for year in pattern for month in months]

# 패턴에 따라 칼럼 이름 정렬
df = df.reindex(columns=sorted_columns)

In [11]:
df.head()

,20년1월_범죄건수,20년2월_범죄건수,20년3월_범죄건수,20년4월_범죄건수,20년5월_범죄건수,20년6월_범죄건수,20년7월_범죄건수,20년8월_범죄건수,20년9월_범죄건수,20년10월_범죄건수,...,22년3월_범죄건수,22년4월_범죄건수,22년5월_범죄건수,22년6월_범죄건수,22년7월_범죄건수,22년8월_범죄건수,22년9월_범죄건수,22년10월_범죄건수,22년11월_범죄건수,22년12월_범죄건수
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,715,508,679,604,582,631,641,651,296,519,...,518,534,617,579,624,643,549,580,706,630
강동구,332,287,283,264,299,330,373,357,89,310,...,244,273,313,325,360,329,298,270,450,303
강북구,258,242,258,252,243,253,265,222,210,212,...,204,231,302,295,230,262,244,240,241,214
강서구,443,373,421,330,371,371,369,380,139,377,...,357,379,382,416,465,433,368,471,434,343
관악구,466,365,396,456,459,441,494,2409,252,469,...,310,392,355,423,463,489,329,395,600,461


In [14]:
data = df.T
data.to_csv('시계열분석_전처리.csv', encoding='cp949')
data.head()

자치구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
20년1월_범죄건수,715,332,258,443,466,344,394,222,274,188,...,239,216,515,255,489,277,329,296,416,280
20년2월_범죄건수,508,287,242,373,365,251,262,264,274,166,...,164,186,337,265,395,209,242,228,267,275
20년3월_범죄건수,679,283,258,421,396,297,322,239,266,169,...,173,220,473,239,440,282,277,310,326,333
20년4월_범죄건수,604,264,252,330,456,276,310,193,306,176,...,194,217,475,238,412,291,259,211,230,292
20년5월_범죄건수,582,299,243,371,459,290,386,233,370,170,...,177,185,515,244,493,266,259,253,258,301


In [122]:
df = pd.read_csv('시계열분석_전처리.csv', encoding='cp949')
df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,20년1월_범죄건수,715,332,258,443,466,344,394,222,274,...,239,216,515,255,489,277,329,296,416,280
1,20년2월_범죄건수,508,287,242,373,365,251,262,264,274,...,164,186,337,265,395,209,242,228,267,275
2,20년3월_범죄건수,679,283,258,421,396,297,322,239,266,...,173,220,473,239,440,282,277,310,326,333
3,20년4월_범죄건수,604,264,252,330,456,276,310,193,306,...,194,217,475,238,412,291,259,211,230,292
4,20년5월_범죄건수,582,299,243,371,459,290,386,233,370,...,177,185,515,244,493,266,259,253,258,301


In [123]:
# '범죄건수' 문자열 삭제
df['Date'] = df['Date'].str.replace('_범죄건수', '')

df['Date'] = df['Date'].str.replace(r'(\d+)년(\d+)월', r'20\1\2', regex=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.strftime('%Y%m')

df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,202001,715,332,258,443,466,344,394,222,274,...,239,216,515,255,489,277,329,296,416,280
1,202002,508,287,242,373,365,251,262,264,274,...,164,186,337,265,395,209,242,228,267,275
2,202003,679,283,258,421,396,297,322,239,266,...,173,220,473,239,440,282,277,310,326,333
3,202004,604,264,252,330,456,276,310,193,306,...,194,217,475,238,412,291,259,211,230,292
4,202005,582,299,243,371,459,290,386,233,370,...,177,185,515,244,493,266,259,253,258,301


In [84]:
gu_name = ['강남구', 'Date']

last_3year = list()
for i in range(2023, 2027):
    for j in range(1, 13):
        last_3year.append(['%04d-%02d' % (i,j)])
        
# for i in range(2026, 2027):
#     for j in range(1, 5):
#         last_3year.append(['%04d-%02d' % (i,j)])
        
last_3year = pd.DataFrame(last_3year, columns = ['ds'])
last_3year['ds']= pd.to_datetime(last_3year['ds'])

In [85]:
pd.to_datetime(df['Date'], format='%Y%m')

0    2020-01-01
1    2020-02-01
2    2020-03-01
3    2020-04-01
4    2020-05-01
5    2020-06-01
6    2020-07-01
7    2020-08-01
8    2020-09-01
9    2020-10-01
10   2020-11-01
11   2020-12-01
12   2021-01-01
13   2021-02-01
14   2021-03-01
15   2021-04-01
16   2021-05-01
17   2021-06-01
18   2021-07-01
19   2021-08-01
20   2021-09-01
21   2021-10-01
22   2021-11-01
23   2021-12-01
24   2022-01-01
25   2022-02-01
26   2022-03-01
27   2022-04-01
28   2022-05-01
29   2022-06-01
30   2022-07-01
31   2022-08-01
32   2022-09-01
33   2022-10-01
34   2022-11-01
35   2022-12-01
Name: Date, dtype: datetime64[ns]

In [86]:
for i in range(len(df['Date'])):
    date_time_str = df['Date'][i]
    
    date = datetime.datetime.strptime(str(date_time_str), '%Y%m')  #datetime.datetime.strptime(date, "%Y-%m-%d")
    
    df['Date'][i] = date

In [87]:
gangnam_predict = pd.DataFrame()

for i,name in enumerate(gu_name):
    data = pd.DataFrame()
    data = df[[name,'Date']]
    data.columns = ['y','ds']
#    data['ds'] = pd.to_numeric(data['ds'])
    data['ds'] = pd.to_datetime(data['ds'])
#    data = df.apply(pd.to_numeric)

    model = Prophet().fit(data)
#    model.fit(data)
    print(i,"model_fit")

    forecast = model.predict(last_3year)
    print(i,"model_predict")
    gangnam_predict = pd.concat([gangnam_predict, forecast['yhat']], ignore_index = True, axis=1)

04:45:58 - cmdstanpy - INFO - Chain [1] start processing
04:45:59 - cmdstanpy - INFO - Chain [1] done processing


0 model_fit
0 model_predict


TypeError: Invalid object type at position 0

In [88]:
gangnam_predict = pd.concat([gangnam_predict, forecast['ds']], ignore_index = True, axis=1)

gangnam_predict.columns = ['강남구', 'Date']

In [90]:
gangnam_predict.to_csv('gangnam_all_predict.csv', encoding='cp949',index = False)

In [124]:
gu_name = ['강동구', 'Date']

last_3year = list()
for i in range(2023, 2027):
    for j in range(1, 13):
        last_3year.append(['%04d-%02d' % (i,j)])
        
        
last_3year = pd.DataFrame(last_3year, columns = ['ds'])
last_3year['ds']= pd.to_datetime(last_3year['ds'])

for i in range(len(df['Date'])):
    date_time_str = df['Date'][i]
    
    date = datetime.datetime.strptime(str(date_time_str), '%Y%m')  #datetime.datetime.strptime(date, "%Y-%m-%d")
    
    df['Date'][i] = date

gangdong_predict = pd.DataFrame()

for i,name in enumerate(gu_name):
    data = pd.DataFrame()
    data = df[[name,'Date']]
    data.columns = ['y','ds']
    data['ds'] = pd.to_datetime(data['ds'])

    model = Prophet().fit(data)
    print(i,"model_fit")
    forecast = model.predict(last_3year)
    print(i,"model_predict")
    gangdong_predict = pd.concat([gangdong_predict, forecast['yhat']], ignore_index = True, axis=1)

04:57:56 - cmdstanpy - INFO - Chain [1] start processing
04:57:57 - cmdstanpy - INFO - Chain [1] done processing


0 model_fit
0 model_predict


TypeError: Invalid object type at position 0

In [125]:
gangdong_predict = pd.concat([gangdong_predict, forecast['ds']], ignore_index = True, axis=1)

gangdong_predict.columns = ['강동구', 'Date']

gangdong_predict.to_csv('gangndong_all_predict.csv', encoding='cp949',index = False)